In [305]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.model_selection import TimeSeriesSplit
from sklearn.preprocessing import LabelEncoder

In [306]:
df = pd.read_csv("dataset_mood_smartphone.csv")
df['time'] = pd.to_datetime(df['time'])
df['date'] = df['time'].dt.date
df.rename(columns={'Unnamed: 0': 'index'}, inplace=True)

df

,index,id,time,variable,value,date
0,1,AS14.01,2014-02-26 13:00:00.000,mood,6.000,2014-02-26
1,2,AS14.01,2014-02-26 15:00:00.000,mood,6.000,2014-02-26
2,3,AS14.01,2014-02-26 18:00:00.000,mood,6.000,2014-02-26
3,4,AS14.01,2014-02-26 21:00:00.000,mood,7.000,2014-02-26
4,5,AS14.01,2014-02-27 09:00:00.000,mood,6.000,2014-02-27
...,...,...,...,...,...,...
376907,2770399,AS14.30,2014-04-11 07:51:16.948,appCat.weather,8.032,2014-04-11
376908,2772465,AS14.30,2014-04-19 11:00:32.747,appCat.weather,3.008,2014-04-19
376909,2774026,AS14.30,2014-04-26 10:19:07.434,appCat.weather,7.026,2014-04-26
376910,2774133,AS14.30,2014-04-27 00:44:48.450,appCat.weather,23.033,2014-04-27


In [307]:
df_filtered = df[df['variable'].isin(['sms', 'call'])]

# Aggregate counts by individual and event type
counts = df_filtered.groupby(['id', 'variable'])['value'].size().reset_index(name='count')

# Determine the IQR for outlier detection
Q1 = counts['count'].quantile(0.25)
Q3 = counts['count'].quantile(0.75)
IQR = Q3 - Q1
outliers_threshold_low = Q1 - 1.5 * IQR
outliers_threshold_high = Q3 + 1.5 * IQR

# Identify outliers
outliers = counts[(counts['count'] < outliers_threshold_low) | (counts['count'] > outliers_threshold_high)]

In [308]:
# Filter for 'sms' and 'call' events
df_sms_call = df[df['variable'].isin(['sms', 'call'])]

# Group by individual ('id'), variable, and date to count daily events
daily_counts = df_sms_call.groupby(['id', 'variable', df_sms_call['time'].dt.date]).size().reset_index(name='daily_count')


In [309]:
# Calculate IQR for daily counts
Q1 = daily_counts['daily_count'].quantile(0.25)
Q3 = daily_counts['daily_count'].quantile(0.75)
IQR = Q3 - Q1

# Define outliers as those beyond 1.5 times the IQR from the quartiles
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

# Identify outlier rows based on daily counts
outliers = daily_counts[(daily_counts['daily_count'] < lower_bound) | (daily_counts['daily_count'] > upper_bound)]


In [310]:
nan_rows_arousal = df[(df['variable'] == 'circumplex.arousal') & (df['value'].isna())].index
nan_rows_valence = df[(df['variable'] == 'circumplex.valence') & (df['value'].isna())].index
nan_rows_activity = df[(df['variable'] == 'activity') & (df['value'].isna())].index


# Combine the indices of rows with NaN values for arousal and valence
nan_rows_combined = nan_rows_arousal.union(nan_rows_valence)

In [311]:
# List of variables/categories to check for negative values, excluding mood, arousal, and valence
variables_to_check = [variable for variable in df['variable'].unique() if variable not in ['mood', 'circumplex.arousal', 'circumplex.valence']]

# Check for negative values in the remaining variables
negative_values_check = {variable: (df[df['variable'] == variable]['value'] < 0).any() for variable in variables_to_check}

negative_values_check

# Identify rows with negative values in appCat.builtin and appCat.entertainment in the original dataset
negative_values_builtin = df[(df['variable'] == 'appCat.builtin') & (df['value'] < 0)].index
negative_values_entertainment = df[(df['variable'] == 'appCat.entertainment') & (df['value'] < 0)].index

#neg combined 
neg = negative_values_builtin.union(negative_values_entertainment)
# Combine the indices of rows with negative values for appCat.builtin and appCat.entertainment
# with previously identified NaN rows for removal
remove_combined = nan_rows_combined.union(negative_values_builtin).union(negative_values_entertainment)

df_negative = df.loc[neg]

In [312]:
combined = df.loc[remove_combined]
combined
df1 = df.drop(combined.index)

In [313]:
# Initialize a DataFrame to store outliers
outliers_df = pd.DataFrame()

variables_for_box_plots_all = [
    'screen', 'appCat.builtin', 'appCat.communication', 'appCat.entertainment',
    'appCat.finance', 'appCat.game', 'appCat.office', 'appCat.other', 'appCat.social',
    'appCat.travel', 'appCat.unknown', 'appCat.utilities', 'appCat.weather'
]
# Iterate over each variable to remove outliers, saving them first
for variable in variables_for_box_plots_all:
    # Isolate the current variable's data
    var_df = df1[df1['variable'] == variable]
    
    # Calculate IQR and determine bounds for outliers
    Q1 = var_df['value'].quantile(0.25)
    Q3 = var_df['value'].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    
    # Identifying outliers
    outliers = var_df[(var_df['value'] < lower_bound) | (var_df['value'] > upper_bound)]
    
    # Append outliers to the outliers_df DataFrame
    outliers_df = pd.concat([outliers_df, outliers], ignore_index=True)
    
    # Identifying indexes of rows that are not outliers to keep in the original dataframe
    non_outliers_index = var_df[(var_df['value'] >= lower_bound) & (var_df['value'] <= upper_bound)].index
    
    # Update df to only include rows that are not outliers for the current variable
    df1 = df1[(df1.index.isin(non_outliers_index)) | (df1['variable'] != variable)]


In [314]:
score_variables = ["mood", "circumplex.arousal", "circumplex.valence", "activity"]

# Creating a dataset with only the selected variables
df_score = df1[df1['variable'].isin(score_variables)]

# Creating another dataset with the rest of the variables
df_machine = df1[~df1['variable'].isin(score_variables)]

df_score['date'] = df_score['time'].dt.date
df_machine['date'] = df_machine['time'].dt.date


/var/folders/_1/g4tvh5tn2wb8t1qyc72n4cdr0000gn/T/ipykernel_31494/3639623860.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_score['date'] = df_score['time'].dt.date
/var/folders/_1/g4tvh5tn2wb8t1qyc72n4cdr0000gn/T/ipykernel_31494/3639623860.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_machine['date'] = df_machine['time'].dt.date


In [315]:
df_scores_daily = df_score.drop('time', axis = 1)
df_machine_daily = df_machine.drop('time', axis = 1)

In [316]:
grouped_scores = df_scores_daily.groupby(['id', 'date', 'variable'])['value'].mean().reset_index()
grouped_scores
grouped_times = df_machine_daily.groupby(['id', 'date', 'variable'])['value'].sum().reset_index()
grouped_times

,id,date,variable,value
0,AS14.01,2014-02-17,call,2.000000
1,AS14.01,2014-02-18,call,1.000000
2,AS14.01,2014-02-19,call,7.000000
3,AS14.01,2014-02-19,sms,2.000000
4,AS14.01,2014-02-20,call,2.000000
...,...,...,...,...
10255,AS14.33,2014-05-30,appCat.travel,915.714000
10256,AS14.33,2014-05-30,appCat.unknown,8.072000
10257,AS14.33,2014-05-30,appCat.utilities,155.922000
10258,AS14.33,2014-05-30,call,4.000000


In [317]:
scores_pivot_df = df_scores_daily.pivot_table(index=['id','date'], columns='variable', values='value', aggfunc='mean').reset_index()
scores_pivot_df

variable,id,date,activity,circumplex.arousal,circumplex.valence,mood
0,AS14.01,2014-02-26,NaN,-0.250000,0.750000,6.250000
1,AS14.01,2014-02-27,NaN,0.000000,0.333333,6.333333
2,AS14.01,2014-03-20,0.081548,NaN,NaN,NaN
3,AS14.01,2014-03-21,0.134050,0.200000,0.200000,6.200000
4,AS14.01,2014-03-22,0.236880,0.600000,0.500000,6.400000
...,...,...,...,...,...,...
1301,AS14.33,2014-05-27,0.012704,-0.600000,0.400000,6.200000
1302,AS14.33,2014-05-28,0.103301,0.000000,1.200000,8.200000
1303,AS14.33,2014-05-29,0.169354,-1.333333,1.000000,7.000000
1304,AS14.33,2014-05-30,0.192901,-0.800000,-0.400000,6.800000


In [318]:
time_pivot_df = df_machine_daily.pivot_table(index=['id','date'], columns='variable', values='value', aggfunc='sum').reset_index()

In [319]:
app_categories_columns = [
    "appCat.builtin", "appCat.communication", "appCat.entertainment",
    "appCat.finance", "appCat.game", "appCat.office", "appCat.other",
    "appCat.social", "appCat.travel", "appCat.unknown", "appCat.utilities",
    "appCat.weather"
]

# Ensure the DataFrame has these columns; this prevents KeyError if some columns don't exist
existing_app_columns = [col for col in app_categories_columns if col in time_pivot_df.columns]

# Replace NaN values with 0 for the specified app category columns
time_pivot_df[existing_app_columns] = time_pivot_df[existing_app_columns].fillna(0)
time_pivot_df



variable,id,date,appCat.builtin,appCat.communication,appCat.entertainment,appCat.finance,appCat.game,appCat.office,appCat.other,appCat.social,appCat.travel,appCat.unknown,appCat.utilities,appCat.weather,call,screen,sms
0,AS14.01,2014-02-17,0.000,0.000,0.000,0.0,0.0,0.000,0.000,0.000,0.000,0.000,0.000,0.0,2.0,NaN,NaN
1,AS14.01,2014-02-18,0.000,0.000,0.000,0.0,0.0,0.000,0.000,0.000,0.000,0.000,0.000,0.0,1.0,NaN,NaN
2,AS14.01,2014-02-19,0.000,0.000,0.000,0.0,0.0,0.000,0.000,0.000,0.000,0.000,0.000,0.0,7.0,NaN,2.0
3,AS14.01,2014-02-20,0.000,0.000,0.000,0.0,0.0,0.000,0.000,0.000,0.000,0.000,0.000,0.0,2.0,NaN,3.0
4,AS14.01,2014-02-21,0.000,0.000,0.000,0.0,0.0,0.000,0.000,0.000,0.000,0.000,0.000,0.0,NaN,NaN,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1910,AS14.33,2014-05-26,571.902,750.571,79.963,0.0,0.0,0.000,140.446,1508.191,0.000,0.000,0.000,0.0,10.0,1118.782999,3.0
1911,AS14.33,2014-05-27,269.550,771.182,32.137,0.0,0.0,0.000,57.535,1252.079,0.000,0.000,56.173,0.0,1.0,1508.516001,2.0
1912,AS14.33,2014-05-28,1093.741,1057.568,153.121,0.0,0.0,38.262,208.962,1817.695,0.000,0.000,30.666,0.0,10.0,3414.742998,1.0
1913,AS14.33,2014-05-29,248.063,198.112,5.018,0.0,0.0,0.000,29.202,377.179,0.939,0.000,3.199,0.0,5.0,395.046000,1.0


In [320]:
merged_df = pd.merge(scores_pivot_df, time_pivot_df, on=['id','date'], how='inner')
merged_df

variable,id,date,activity,circumplex.arousal,circumplex.valence,mood,appCat.builtin,appCat.communication,appCat.entertainment,appCat.finance,...,appCat.office,appCat.other,appCat.social,appCat.travel,appCat.unknown,appCat.utilities,appCat.weather,call,screen,sms
0,AS14.01,2014-02-26,NaN,-0.250000,0.75,6.25,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,1.0,NaN,2.0
1,AS14.01,2014-03-20,0.081548,NaN,NaN,NaN,121.466,980.911,42.593,0.000,...,0.000,11.345,621.094,0.000,45.173,21.074,0.000,1.0,128.843000,NaN
2,AS14.01,2014-03-21,0.134050,0.200000,0.20,6.20,1090.617,2850.042,100.898,49.544,...,5.479,50.465,3121.747,167.373,0.000,229.138,0.000,6.0,2867.963000,NaN
3,AS14.01,2014-03-22,0.236880,0.600000,0.50,6.40,476.737,1946.683,4.010,21.076,...,0.000,60.565,439.632,37.305,0.000,53.188,0.000,3.0,3038.464000,1.0
4,AS14.01,2014-03-23,0.142741,0.200000,0.80,6.80,540.915,2680.535,4.028,43.403,...,0.000,40.604,649.414,0.000,0.000,30.086,30.386,NaN,2780.277001,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1243,AS14.33,2014-05-26,0.115201,-0.400000,0.00,5.40,571.902,750.571,79.963,0.000,...,0.000,140.446,1508.191,0.000,0.000,0.000,0.000,10.0,1118.782999,3.0
1244,AS14.33,2014-05-27,0.012704,-0.600000,0.40,6.20,269.550,771.182,32.137,0.000,...,0.000,57.535,1252.079,0.000,0.000,56.173,0.000,1.0,1508.516001,2.0
1245,AS14.33,2014-05-28,0.103301,0.000000,1.20,8.20,1093.741,1057.568,153.121,0.000,...,38.262,208.962,1817.695,0.000,0.000,30.666,0.000,10.0,3414.742998,1.0
1246,AS14.33,2014-05-29,0.169354,-1.333333,1.00,7.00,248.063,198.112,5.018,0.000,...,0.000,29.202,377.179,0.939,0.000,3.199,0.000,5.0,395.046000,1.0


NOW SPLIT INTO TRAIN TEST SETS, THEN ADD ALL INTERPOLATION STUFF TO BOTH

In [321]:
#tscv = TimeSeriesSplit(n_splits=2) #ONLY 1 SPLIT = change to 5 for kfoldcross
train_df = merged_df

In [322]:
columns_to_interpolate = ['call',  'sms']
train_df[columns_to_interpolate] = train_df[columns_to_interpolate].interpolate(method='linear', limit_direction='forward', axis=0)
mean_activity_per_id = train_df.groupby('id')['activity'].mean()
train_df['activity'] = train_df.apply(
    lambda row: mean_activity_per_id[row['id']] if pd.isna(row['activity']) else row['activity'],
    axis=1
)
#test_df[columns_to_interpolate] = test_df[columns_to_interpolate].interpolate(method='linear', limit_direction='forward', axis=0)
train_df

variable,id,date,activity,circumplex.arousal,circumplex.valence,mood,appCat.builtin,appCat.communication,appCat.entertainment,appCat.finance,...,appCat.office,appCat.other,appCat.social,appCat.travel,appCat.unknown,appCat.utilities,appCat.weather,call,screen,sms
0,AS14.01,2014-02-26,0.090464,-0.250000,0.75,6.25,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,1.0,NaN,2.000000
1,AS14.01,2014-03-20,0.081548,NaN,NaN,NaN,121.466,980.911,42.593,0.000,...,0.000,11.345,621.094,0.000,45.173,21.074,0.000,1.0,128.843000,1.666667
2,AS14.01,2014-03-21,0.134050,0.200000,0.20,6.20,1090.617,2850.042,100.898,49.544,...,5.479,50.465,3121.747,167.373,0.000,229.138,0.000,6.0,2867.963000,1.333333
3,AS14.01,2014-03-22,0.236880,0.600000,0.50,6.40,476.737,1946.683,4.010,21.076,...,0.000,60.565,439.632,37.305,0.000,53.188,0.000,3.0,3038.464000,1.000000
4,AS14.01,2014-03-23,0.142741,0.200000,0.80,6.80,540.915,2680.535,4.028,43.403,...,0.000,40.604,649.414,0.000,0.000,30.086,30.386,6.5,2780.277001,1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1243,AS14.33,2014-05-26,0.115201,-0.400000,0.00,5.40,571.902,750.571,79.963,0.000,...,0.000,140.446,1508.191,0.000,0.000,0.000,0.000,10.0,1118.782999,3.000000
1244,AS14.33,2014-05-27,0.012704,-0.600000,0.40,6.20,269.550,771.182,32.137,0.000,...,0.000,57.535,1252.079,0.000,0.000,56.173,0.000,1.0,1508.516001,2.000000
1245,AS14.33,2014-05-28,0.103301,0.000000,1.20,8.20,1093.741,1057.568,153.121,0.000,...,38.262,208.962,1817.695,0.000,0.000,30.666,0.000,10.0,3414.742998,1.000000
1246,AS14.33,2014-05-29,0.169354,-1.333333,1.00,7.00,248.063,198.112,5.018,0.000,...,0.000,29.202,377.179,0.939,0.000,3.199,0.000,5.0,395.046000,1.000000


In [323]:
columns_to_interpolate = ['call', 'sms', 'activity']
train_df[columns_to_interpolate] = train_df.groupby('id')[columns_to_interpolate].transform(lambda group: group.interpolate(method='linear', limit_direction='forward', axis=0))
#test_df[columns_to_interpolate] = test_df.groupby('id')[columns_to_interpolate].transform(lambda group: group.interpolate(method='linear', limit_direction='forward', axis=0))

train_df

variable,id,date,activity,circumplex.arousal,circumplex.valence,mood,appCat.builtin,appCat.communication,appCat.entertainment,appCat.finance,...,appCat.office,appCat.other,appCat.social,appCat.travel,appCat.unknown,appCat.utilities,appCat.weather,call,screen,sms
0,AS14.01,2014-02-26,0.090464,-0.250000,0.75,6.25,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,1.0,NaN,2.000000
1,AS14.01,2014-03-20,0.081548,NaN,NaN,NaN,121.466,980.911,42.593,0.000,...,0.000,11.345,621.094,0.000,45.173,21.074,0.000,1.0,128.843000,1.666667
2,AS14.01,2014-03-21,0.134050,0.200000,0.20,6.20,1090.617,2850.042,100.898,49.544,...,5.479,50.465,3121.747,167.373,0.000,229.138,0.000,6.0,2867.963000,1.333333
3,AS14.01,2014-03-22,0.236880,0.600000,0.50,6.40,476.737,1946.683,4.010,21.076,...,0.000,60.565,439.632,37.305,0.000,53.188,0.000,3.0,3038.464000,1.000000
4,AS14.01,2014-03-23,0.142741,0.200000,0.80,6.80,540.915,2680.535,4.028,43.403,...,0.000,40.604,649.414,0.000,0.000,30.086,30.386,6.5,2780.277001,1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1243,AS14.33,2014-05-26,0.115201,-0.400000,0.00,5.40,571.902,750.571,79.963,0.000,...,0.000,140.446,1508.191,0.000,0.000,0.000,0.000,10.0,1118.782999,3.000000
1244,AS14.33,2014-05-27,0.012704,-0.600000,0.40,6.20,269.550,771.182,32.137,0.000,...,0.000,57.535,1252.079,0.000,0.000,56.173,0.000,1.0,1508.516001,2.000000
1245,AS14.33,2014-05-28,0.103301,0.000000,1.20,8.20,1093.741,1057.568,153.121,0.000,...,38.262,208.962,1817.695,0.000,0.000,30.666,0.000,10.0,3414.742998,1.000000
1246,AS14.33,2014-05-29,0.169354,-1.333333,1.00,7.00,248.063,198.112,5.018,0.000,...,0.000,29.202,377.179,0.939,0.000,3.199,0.000,5.0,395.046000,1.000000


In [324]:
columns_to_check = ['mood']


train_df_cleaned = train_df.dropna(subset=columns_to_check)
#test_df = test_df.dropna(subset=columns_to_check)
train_df_cleaned

variable,id,date,activity,circumplex.arousal,circumplex.valence,mood,appCat.builtin,appCat.communication,appCat.entertainment,appCat.finance,...,appCat.office,appCat.other,appCat.social,appCat.travel,appCat.unknown,appCat.utilities,appCat.weather,call,screen,sms
0,AS14.01,2014-02-26,0.090464,-0.250000,0.75,6.25,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,1.0,NaN,2.000000
2,AS14.01,2014-03-21,0.134050,0.200000,0.20,6.20,1090.617,2850.042,100.898,49.544,...,5.479,50.465,3121.747,167.373,0.000,229.138,0.000,6.0,2867.963000,1.333333
3,AS14.01,2014-03-22,0.236880,0.600000,0.50,6.40,476.737,1946.683,4.010,21.076,...,0.000,60.565,439.632,37.305,0.000,53.188,0.000,3.0,3038.464000,1.000000
4,AS14.01,2014-03-23,0.142741,0.200000,0.80,6.80,540.915,2680.535,4.028,43.403,...,0.000,40.604,649.414,0.000,0.000,30.086,30.386,6.5,2780.277001,1.000000
5,AS14.01,2014-03-24,0.078961,0.800000,0.00,6.00,731.456,5070.628,70.289,34.106,...,3.010,66.558,1362.785,35.005,0.000,10.064,0.000,10.0,5953.697001,1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1243,AS14.33,2014-05-26,0.115201,-0.400000,0.00,5.40,571.902,750.571,79.963,0.000,...,0.000,140.446,1508.191,0.000,0.000,0.000,0.000,10.0,1118.782999,3.000000
1244,AS14.33,2014-05-27,0.012704,-0.600000,0.40,6.20,269.550,771.182,32.137,0.000,...,0.000,57.535,1252.079,0.000,0.000,56.173,0.000,1.0,1508.516001,2.000000
1245,AS14.33,2014-05-28,0.103301,0.000000,1.20,8.20,1093.741,1057.568,153.121,0.000,...,38.262,208.962,1817.695,0.000,0.000,30.666,0.000,10.0,3414.742998,1.000000
1246,AS14.33,2014-05-29,0.169354,-1.333333,1.00,7.00,248.063,198.112,5.018,0.000,...,0.000,29.202,377.179,0.939,0.000,3.199,0.000,5.0,395.046000,1.000000


In [325]:
not_null_columns = [
    "circumplex.valence"
]

# Ensure the DataFrame has these columns; this prevents KeyError if some columns don't exist
existing_app_columns = [col for col in not_null_columns if col in train_df_cleaned.columns]

# Replace NaN values with 0 for the specified app category columns
train_df_cleaned[existing_app_columns] = train_df_cleaned[existing_app_columns].fillna(0)
#test_df[existing_app_columns] = test_df[existing_app_columns].fillna(0)
train_df_cleaned


/var/folders/_1/g4tvh5tn2wb8t1qyc72n4cdr0000gn/T/ipykernel_31494/4185145295.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_df_cleaned[existing_app_columns] = train_df_cleaned[existing_app_columns].fillna(0)


variable,id,date,activity,circumplex.arousal,circumplex.valence,mood,appCat.builtin,appCat.communication,appCat.entertainment,appCat.finance,...,appCat.office,appCat.other,appCat.social,appCat.travel,appCat.unknown,appCat.utilities,appCat.weather,call,screen,sms
0,AS14.01,2014-02-26,0.090464,-0.250000,0.75,6.25,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,1.0,NaN,2.000000
2,AS14.01,2014-03-21,0.134050,0.200000,0.20,6.20,1090.617,2850.042,100.898,49.544,...,5.479,50.465,3121.747,167.373,0.000,229.138,0.000,6.0,2867.963000,1.333333
3,AS14.01,2014-03-22,0.236880,0.600000,0.50,6.40,476.737,1946.683,4.010,21.076,...,0.000,60.565,439.632,37.305,0.000,53.188,0.000,3.0,3038.464000,1.000000
4,AS14.01,2014-03-23,0.142741,0.200000,0.80,6.80,540.915,2680.535,4.028,43.403,...,0.000,40.604,649.414,0.000,0.000,30.086,30.386,6.5,2780.277001,1.000000
5,AS14.01,2014-03-24,0.078961,0.800000,0.00,6.00,731.456,5070.628,70.289,34.106,...,3.010,66.558,1362.785,35.005,0.000,10.064,0.000,10.0,5953.697001,1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1243,AS14.33,2014-05-26,0.115201,-0.400000,0.00,5.40,571.902,750.571,79.963,0.000,...,0.000,140.446,1508.191,0.000,0.000,0.000,0.000,10.0,1118.782999,3.000000
1244,AS14.33,2014-05-27,0.012704,-0.600000,0.40,6.20,269.550,771.182,32.137,0.000,...,0.000,57.535,1252.079,0.000,0.000,56.173,0.000,1.0,1508.516001,2.000000
1245,AS14.33,2014-05-28,0.103301,0.000000,1.20,8.20,1093.741,1057.568,153.121,0.000,...,38.262,208.962,1817.695,0.000,0.000,30.666,0.000,10.0,3414.742998,1.000000
1246,AS14.33,2014-05-29,0.169354,-1.333333,1.00,7.00,248.063,198.112,5.018,0.000,...,0.000,29.202,377.179,0.939,0.000,3.199,0.000,5.0,395.046000,1.000000


In [326]:
app_categories_columns = [
    "appCat.builtin", "appCat.communication", "appCat.entertainment",
    "appCat.finance", "appCat.game", "appCat.office", "appCat.other",
    "appCat.social", "appCat.travel", "appCat.unknown", "appCat.utilities",
    "appCat.weather",
]

# Ensure the DataFrame has these columns; this prevents KeyError if some columns don't exist
existing_app_columns = [col for col in app_categories_columns if col in train_df_cleaned.columns]

# Replace NaN values with 0 for the specified app category columns
train_df_cleaned[existing_app_columns] = train_df_cleaned[existing_app_columns].fillna(0)
#test_df[existing_app_columns] = test_df[existing_app_columns].fillna(0)
train_df_cleaned


/var/folders/_1/g4tvh5tn2wb8t1qyc72n4cdr0000gn/T/ipykernel_31494/1844166882.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_df_cleaned[existing_app_columns] = train_df_cleaned[existing_app_columns].fillna(0)


variable,id,date,activity,circumplex.arousal,circumplex.valence,mood,appCat.builtin,appCat.communication,appCat.entertainment,appCat.finance,...,appCat.office,appCat.other,appCat.social,appCat.travel,appCat.unknown,appCat.utilities,appCat.weather,call,screen,sms
0,AS14.01,2014-02-26,0.090464,-0.250000,0.75,6.25,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,1.0,NaN,2.000000
2,AS14.01,2014-03-21,0.134050,0.200000,0.20,6.20,1090.617,2850.042,100.898,49.544,...,5.479,50.465,3121.747,167.373,0.000,229.138,0.000,6.0,2867.963000,1.333333
3,AS14.01,2014-03-22,0.236880,0.600000,0.50,6.40,476.737,1946.683,4.010,21.076,...,0.000,60.565,439.632,37.305,0.000,53.188,0.000,3.0,3038.464000,1.000000
4,AS14.01,2014-03-23,0.142741,0.200000,0.80,6.80,540.915,2680.535,4.028,43.403,...,0.000,40.604,649.414,0.000,0.000,30.086,30.386,6.5,2780.277001,1.000000
5,AS14.01,2014-03-24,0.078961,0.800000,0.00,6.00,731.456,5070.628,70.289,34.106,...,3.010,66.558,1362.785,35.005,0.000,10.064,0.000,10.0,5953.697001,1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1243,AS14.33,2014-05-26,0.115201,-0.400000,0.00,5.40,571.902,750.571,79.963,0.000,...,0.000,140.446,1508.191,0.000,0.000,0.000,0.000,10.0,1118.782999,3.000000
1244,AS14.33,2014-05-27,0.012704,-0.600000,0.40,6.20,269.550,771.182,32.137,0.000,...,0.000,57.535,1252.079,0.000,0.000,56.173,0.000,1.0,1508.516001,2.000000
1245,AS14.33,2014-05-28,0.103301,0.000000,1.20,8.20,1093.741,1057.568,153.121,0.000,...,38.262,208.962,1817.695,0.000,0.000,30.666,0.000,10.0,3414.742998,1.000000
1246,AS14.33,2014-05-29,0.169354,-1.333333,1.00,7.00,248.063,198.112,5.018,0.000,...,0.000,29.202,377.179,0.939,0.000,3.199,0.000,5.0,395.046000,1.000000


In [327]:
columns_to_sum = [
    "appCat.builtin", "appCat.communication", "appCat.entertainment",
    "appCat.finance", "appCat.game", "appCat.office", "appCat.other",
    "appCat.social", "appCat.travel", "appCat.unknown", "appCat.utilities",
    "appCat.weather"
]

# Use apply to iterate over rows for rows where 'screen' is NaN
train_df_cleaned.loc[train_df_cleaned['screen'].isna(), 'screen'] = train_df_cleaned[train_df_cleaned['screen'].isna()].apply(
    lambda row: row[columns_to_sum].sum(), axis=1)
#test_df.loc[test_df['screen'].isna(), 'screen'] = test_df[test_df['screen'].isna()].apply(
    #lambda row: row[columns_to_sum].sum(), axis=1)

train_df_cleaned

variable,id,date,activity,circumplex.arousal,circumplex.valence,mood,appCat.builtin,appCat.communication,appCat.entertainment,appCat.finance,...,appCat.office,appCat.other,appCat.social,appCat.travel,appCat.unknown,appCat.utilities,appCat.weather,call,screen,sms
0,AS14.01,2014-02-26,0.090464,-0.250000,0.75,6.25,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,1.0,0.000000,2.000000
2,AS14.01,2014-03-21,0.134050,0.200000,0.20,6.20,1090.617,2850.042,100.898,49.544,...,5.479,50.465,3121.747,167.373,0.000,229.138,0.000,6.0,2867.963000,1.333333
3,AS14.01,2014-03-22,0.236880,0.600000,0.50,6.40,476.737,1946.683,4.010,21.076,...,0.000,60.565,439.632,37.305,0.000,53.188,0.000,3.0,3038.464000,1.000000
4,AS14.01,2014-03-23,0.142741,0.200000,0.80,6.80,540.915,2680.535,4.028,43.403,...,0.000,40.604,649.414,0.000,0.000,30.086,30.386,6.5,2780.277001,1.000000
5,AS14.01,2014-03-24,0.078961,0.800000,0.00,6.00,731.456,5070.628,70.289,34.106,...,3.010,66.558,1362.785,35.005,0.000,10.064,0.000,10.0,5953.697001,1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1243,AS14.33,2014-05-26,0.115201,-0.400000,0.00,5.40,571.902,750.571,79.963,0.000,...,0.000,140.446,1508.191,0.000,0.000,0.000,0.000,10.0,1118.782999,3.000000
1244,AS14.33,2014-05-27,0.012704,-0.600000,0.40,6.20,269.550,771.182,32.137,0.000,...,0.000,57.535,1252.079,0.000,0.000,56.173,0.000,1.0,1508.516001,2.000000
1245,AS14.33,2014-05-28,0.103301,0.000000,1.20,8.20,1093.741,1057.568,153.121,0.000,...,38.262,208.962,1817.695,0.000,0.000,30.666,0.000,10.0,3414.742998,1.000000
1246,AS14.33,2014-05-29,0.169354,-1.333333,1.00,7.00,248.063,198.112,5.018,0.000,...,0.000,29.202,377.179,0.939,0.000,3.199,0.000,5.0,395.046000,1.000000


In [329]:
def custom_3day_avg(series):

    result = np.empty_like(series, dtype=float)
    result[:] = np.nan  # Initialize with NaN
    
    # Iterate over the series using index for access
    for i in range(len(series)):
        if i == 0:
            result[i] = np.nan  # First day is NaN
        elif i == 1:
            result[i] = series.iloc[i-1]  # Second day is the value of the first day
        elif i == 2:
            result[i] = (series.iloc[i-2] + series.iloc[i-1]) / 2  # Third day is the average of the first two days
        else:
            result[i] = (series.iloc[i-3] + series.iloc[i-2] + series.iloc[i-1]) / 3  # From the fourth day, use a regular 3-day average
    
    return pd.Series(result, index=series.index)
    
train_df_cleaned['3_day_avg_mood'] = train_df_cleaned.groupby('id')['mood'].transform(custom_3day_avg)
train_df_cleaned['3_day_avg_activity'] = train_df_cleaned.groupby('id')['activity'].transform(custom_3day_avg)
train_df_cleaned['3_day_avg_mood'] = train_df_cleaned.groupby('id')['3_day_avg_mood'].fillna(method='ffill')
train_df_cleaned['3_day_avg_activity'] = train_df_cleaned.groupby('id')['3_day_avg_activity'].fillna(method='ffill')
#test_df['3_day_avg_mood'] = test_df.groupby('id')['mood'].transform(custom_3day_avg)
#test_df['3_day_avg_activity'] = test_df.groupby('id')['activity'].transform(custom_3day_avg)
#test_df['3_day_avg_mood'] = test_df.groupby('id')['3_day_avg_mood'].fillna(method='ffill')
#test_df['3_day_avg_activity'] = test_df.groupby('id')['3_day_avg_activity'].fillna(method='ffill')
mean_mood_per_id = train_df.groupby('id')['mood'].mean()
train_df_cleaned['3_day_avg_mood'] = train_df_cleaned.apply(
    lambda row: mean_mood_per_id[row['id']] if pd.isna(row['3_day_avg_mood']) else row['3_day_avg_mood'],
    axis=1
)
mean_activity_per_id = train_df.groupby('id')['activity'].mean()
train_df_cleaned['3_day_avg_activity'] = train_df_cleaned.apply(
    lambda row: mean_activity_per_id[row['id']] if pd.isna(row['3_day_avg_activity']) else row['3_day_avg_activity'],
    axis=1
)
train_df_cleaned


/var/folders/_1/g4tvh5tn2wb8t1qyc72n4cdr0000gn/T/ipykernel_31494/664256289.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_df_cleaned['3_day_avg_mood'] = train_df_cleaned.groupby('id')['mood'].transform(custom_3day_avg)
/var/folders/_1/g4tvh5tn2wb8t1qyc72n4cdr0000gn/T/ipykernel_31494/664256289.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_df_cleaned['3_day_avg_activity'] = train_df_cleaned.groupby('id')['activity'].transform(custom_3day_avg)
/var/folders/_1/g4tvh5tn2wb8t1qyc72n4cdr00

variable,id,date,activity,circumplex.arousal,circumplex.valence,mood,appCat.builtin,appCat.communication,appCat.entertainment,appCat.finance,...,appCat.social,appCat.travel,appCat.unknown,appCat.utilities,appCat.weather,call,screen,sms,3_day_avg_mood,3_day_avg_activity
0,AS14.01,2014-02-26,0.090464,-0.250000,0.75,6.25,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,1.0,0.000000,2.000000,7.078261,0.090464
2,AS14.01,2014-03-21,0.134050,0.200000,0.20,6.20,1090.617,2850.042,100.898,49.544,...,3121.747,167.373,0.000,229.138,0.000,6.0,2867.963000,1.333333,6.250000,0.090464
3,AS14.01,2014-03-22,0.236880,0.600000,0.50,6.40,476.737,1946.683,4.010,21.076,...,439.632,37.305,0.000,53.188,0.000,3.0,3038.464000,1.000000,6.225000,0.112257
4,AS14.01,2014-03-23,0.142741,0.200000,0.80,6.80,540.915,2680.535,4.028,43.403,...,649.414,0.000,0.000,30.086,30.386,6.5,2780.277001,1.000000,6.283333,0.153798
5,AS14.01,2014-03-24,0.078961,0.800000,0.00,6.00,731.456,5070.628,70.289,34.106,...,1362.785,35.005,0.000,10.064,0.000,10.0,5953.697001,1.000000,6.466667,0.171224
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1243,AS14.33,2014-05-26,0.115201,-0.400000,0.00,5.40,571.902,750.571,79.963,0.000,...,1508.191,0.000,0.000,0.000,0.000,10.0,1118.782999,3.000000,6.161111,0.065448
1244,AS14.33,2014-05-27,0.012704,-0.600000,0.40,6.20,269.550,771.182,32.137,0.000,...,1252.079,0.000,0.000,56.173,0.000,1.0,1508.516001,2.000000,6.016667,0.096986
1245,AS14.33,2014-05-28,0.103301,0.000000,1.20,8.20,1093.741,1057.568,153.121,0.000,...,1817.695,0.000,0.000,30.666,0.000,10.0,3414.742998,1.000000,5.950000,0.096459
1246,AS14.33,2014-05-29,0.169354,-1.333333,1.00,7.00,248.063,198.112,5.018,0.000,...,377.179,0.939,0.000,3.199,0.000,5.0,395.046000,1.000000,6.600000,0.077069


In [330]:
train_df_cleaned['productivity_to_social_ratio'] = (train_df_cleaned['appCat.finance'] + train_df_cleaned['appCat.office']) +1/ (train_df_cleaned['appCat.entertainment'] + train_df_cleaned['appCat.social']+1)
train_df_cleaned['productivity_attribute'] = train_df_cleaned['appCat.finance'] + train_df_cleaned['appCat.office']
train_df_cleaned['social_attribute'] = train_df_cleaned['appCat.entertainment'] + train_df_cleaned['appCat.social']
#test_df['productivity_to_social_ratio'] = (test_df['appCat.finance'] + test_df['appCat.office']) +1/ (test_df['appCat.entertainment'] + test_df['appCat.social']+1)
#test_df['productivity_attribute'] = test_df['appCat.finance'] + test_df['appCat.office']
#test_df['social_attribute'] = test_df['appCat.entertainment'] + test_df['appCat.social']
train_df_cleaned

/var/folders/_1/g4tvh5tn2wb8t1qyc72n4cdr0000gn/T/ipykernel_31494/2590602958.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_df_cleaned['productivity_to_social_ratio'] = (train_df_cleaned['appCat.finance'] + train_df_cleaned['appCat.office']) +1/ (train_df_cleaned['appCat.entertainment'] + train_df_cleaned['appCat.social']+1)
/var/folders/_1/g4tvh5tn2wb8t1qyc72n4cdr0000gn/T/ipykernel_31494/2590602958.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_df_cleaned['productivity_attribute'] = tr

variable,id,date,activity,circumplex.arousal,circumplex.valence,mood,appCat.builtin,appCat.communication,appCat.entertainment,appCat.finance,...,appCat.utilities,appCat.weather,call,screen,sms,3_day_avg_mood,3_day_avg_activity,productivity_to_social_ratio,productivity_attribute,social_attribute
0,AS14.01,2014-02-26,0.090464,-0.250000,0.75,6.25,0.000,0.000,0.000,0.000,...,0.000,0.000,1.0,0.000000,2.000000,7.078261,0.090464,1.000000,0.000,0.000
2,AS14.01,2014-03-21,0.134050,0.200000,0.20,6.20,1090.617,2850.042,100.898,49.544,...,229.138,0.000,6.0,2867.963000,1.333333,6.250000,0.090464,55.023310,55.023,3222.645
3,AS14.01,2014-03-22,0.236880,0.600000,0.50,6.40,476.737,1946.683,4.010,21.076,...,53.188,0.000,3.0,3038.464000,1.000000,6.225000,0.112257,21.078249,21.076,443.642
4,AS14.01,2014-03-23,0.142741,0.200000,0.80,6.80,540.915,2680.535,4.028,43.403,...,30.086,30.386,6.5,2780.277001,1.000000,6.283333,0.153798,43.404528,43.403,653.442
5,AS14.01,2014-03-24,0.078961,0.800000,0.00,6.00,731.456,5070.628,70.289,34.106,...,10.064,0.000,10.0,5953.697001,1.000000,6.466667,0.171224,37.116697,37.116,1433.074
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1243,AS14.33,2014-05-26,0.115201,-0.400000,0.00,5.40,571.902,750.571,79.963,0.000,...,0.000,0.000,10.0,1118.782999,3.000000,6.161111,0.065448,0.000629,0.000,1588.154
1244,AS14.33,2014-05-27,0.012704,-0.600000,0.40,6.20,269.550,771.182,32.137,0.000,...,56.173,0.000,1.0,1508.516001,2.000000,6.016667,0.096986,0.000778,0.000,1284.216
1245,AS14.33,2014-05-28,0.103301,0.000000,1.20,8.20,1093.741,1057.568,153.121,0.000,...,30.666,0.000,10.0,3414.742998,1.000000,5.950000,0.096459,38.262507,38.262,1970.816
1246,AS14.33,2014-05-29,0.169354,-1.333333,1.00,7.00,248.063,198.112,5.018,0.000,...,3.199,0.000,5.0,395.046000,1.000000,6.600000,0.077069,0.002610,0.000,382.197


In [331]:
# Assuming 'df' is your DataFrame and 'grade' is your continuous variable
train_df_cleaned['mood_quantiles'] = pd.qcut(train_df_cleaned['mood'], q=4, labels=['Q1', 'Q2', 'Q3', 'Q4'])
#test_df['mood_quantiles'] = pd.qcut(test_df['mood'], q=4, labels=['Q1', 'Q2', 'Q3', 'Q4'])
train_df_cleaned


/var/folders/_1/g4tvh5tn2wb8t1qyc72n4cdr0000gn/T/ipykernel_31494/1039805555.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_df_cleaned['mood_quantiles'] = pd.qcut(train_df_cleaned['mood'], q=4, labels=['Q1', 'Q2', 'Q3', 'Q4'])


variable,id,date,activity,circumplex.arousal,circumplex.valence,mood,appCat.builtin,appCat.communication,appCat.entertainment,appCat.finance,...,appCat.weather,call,screen,sms,3_day_avg_mood,3_day_avg_activity,productivity_to_social_ratio,productivity_attribute,social_attribute,mood_quantiles
0,AS14.01,2014-02-26,0.090464,-0.250000,0.75,6.25,0.000,0.000,0.000,0.000,...,0.000,1.0,0.000000,2.000000,7.078261,0.090464,1.000000,0.000,0.000,Q1
2,AS14.01,2014-03-21,0.134050,0.200000,0.20,6.20,1090.617,2850.042,100.898,49.544,...,0.000,6.0,2867.963000,1.333333,6.250000,0.090464,55.023310,55.023,3222.645,Q1
3,AS14.01,2014-03-22,0.236880,0.600000,0.50,6.40,476.737,1946.683,4.010,21.076,...,0.000,3.0,3038.464000,1.000000,6.225000,0.112257,21.078249,21.076,443.642,Q1
4,AS14.01,2014-03-23,0.142741,0.200000,0.80,6.80,540.915,2680.535,4.028,43.403,...,30.386,6.5,2780.277001,1.000000,6.283333,0.153798,43.404528,43.403,653.442,Q2
5,AS14.01,2014-03-24,0.078961,0.800000,0.00,6.00,731.456,5070.628,70.289,34.106,...,0.000,10.0,5953.697001,1.000000,6.466667,0.171224,37.116697,37.116,1433.074,Q1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1243,AS14.33,2014-05-26,0.115201,-0.400000,0.00,5.40,571.902,750.571,79.963,0.000,...,0.000,10.0,1118.782999,3.000000,6.161111,0.065448,0.000629,0.000,1588.154,Q1
1244,AS14.33,2014-05-27,0.012704,-0.600000,0.40,6.20,269.550,771.182,32.137,0.000,...,0.000,1.0,1508.516001,2.000000,6.016667,0.096986,0.000778,0.000,1284.216,Q1
1245,AS14.33,2014-05-28,0.103301,0.000000,1.20,8.20,1093.741,1057.568,153.121,0.000,...,0.000,10.0,3414.742998,1.000000,5.950000,0.096459,38.262507,38.262,1970.816,Q4
1246,AS14.33,2014-05-29,0.169354,-1.333333,1.00,7.00,248.063,198.112,5.018,0.000,...,0.000,5.0,395.046000,1.000000,6.600000,0.077069,0.002610,0.000,382.197,Q2


In [332]:
train_df_cleaned.to_csv('out.csv') 
#test_df.to_csv('test.csv')

#merged_df_2 = pd.concat([train_df_cleaned, test_df])
#merged_df_2.sort_values(by=['id', 'date'])
##merged_df_2.to_csv("out.csv")
train_df_cleaned.isnull().sum()

variable
id                              0
date                            0
activity                        0
circumplex.arousal              0
circumplex.valence              0
mood                            0
appCat.builtin                  0
appCat.communication            0
appCat.entertainment            0
appCat.finance                  0
appCat.game                     0
appCat.office                   0
appCat.other                    0
appCat.social                   0
appCat.travel                   0
appCat.unknown                  0
appCat.utilities                0
appCat.weather                  0
call                            0
screen                          0
sms                             0
3_day_avg_mood                  0
3_day_avg_activity              0
productivity_to_social_ratio    0
productivity_attribute          0
social_attribute                0
mood_quantiles                  0
dtype: int64